In [6]:
import javalang

def segment_java_code(java_code):
    tree = javalang.parse.parse(java_code)
    lines = java_code.splitlines()
    segments = []

    for type in tree.types:
        for member in type.body:
            if not hasattr(member, 'position') or member.position is None:
                continue

            start_line = member.position.line - 1

            open_braces = 0
            end_line = start_line
            for i in range(start_line, len(lines)):
                open_braces += lines[i].count('{')
                open_braces -= lines[i].count('}')
                if open_braces == 0 and '{' in lines[start_line]:
                    end_line = i
                    break

            code_segment = "\n".join(lines[start_line:end_line + 1])

            if isinstance(member, javalang.tree.MethodDeclaration):
                segments.append(("method", member.name, code_segment))
            elif isinstance(member, javalang.tree.ConstructorDeclaration):
                segments.append(("constructor", member.name, code_segment))

    return segments


In [22]:
def is_relevant_segment(code_segment: str, min_lines=4, skip_keywords=None):
    lines = code_segment.strip().splitlines()
    if len(lines) < min_lines:
        print(len(lines))
        return False

    signature = lines[0].strip().lower()
    if skip_keywords:
        for kw in skip_keywords:
            if kw.lower() in signature:
                return False

    return True

In [29]:
java_code = """
/*
 * Licensed to the Apache Software Foundation (ASF) under one or more
 * contributor license agreements.  See the NOTICE file distributed with
 * this work for additional information regarding copyright ownership.
 * The ASF licenses this file to You under the Apache License, Version 2.0
 * (the "License"); you may not use this file except in compliance with
 * the License.  You may obtain a copy of the License at
 *
 *      http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */
public class MyClass {
    public void methodOne() {
    }

    public void methodTwo() {
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");

    }

    private int methodTwo(int x) {
        return x * 2;}

    public MyClass() {
        // constructor
    }

    interface MyInterface {
        void doSomething();
    }
}
"""

segments = segment_java_code(java_code)

for seg_type, name, code in segments:
    print(code)
    print()


    public void methodOne() {
    }

    public void methodTwo() {
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");

    }

    private int methodTwo(int x) {
        return x * 2;}

    public MyClass() {
        // constructor
    }



In [30]:
filtered_segments = []
for seg_type, name, code in segments:
    if is_relevant_segment(code, min_lines=3, skip_keywords=['toString', 'hashCode', 'equals']):
        filtered_segments.append((seg_type, name, code))

2
2


In [31]:
for seg_type, name, code in filtered_segments:
    print(code)
    print()

    public void methodTwo() {
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");
    System.out.println("Hello");

    }

    public MyClass() {
        // constructor
    }

